In [17]:
try:
    from variables import Variable
except:
    from AutoDiff.variables import Variable
try:
    from vectorize_func import vectorize_variable
except:
    from AutoDiff.vectorize_func import vectorize_variable
try:
    import AD_numpy as anp
except:
    import AutoDiff.AD_numpy as anp
import numpy as np

In [18]:
#@vectorize_variable
#def vec_fn(x, y):
#    f1 = anp.cos(x) + anp.sin(y)
#    f2 = x**2 + y**2
#    return np.array([f1,f2])

In [19]:
def newton_method_scalar(function, list_variables, threshold, max_iter, verbose=True):
    nums_iteration = 1
    key_val_dict = {}
    while True:
        # This line is the only difference between two newton_method_scalar functions
        function = x**2 + y**2
        value_f = function.val
        for var in list_variables:
            var.val = var.val - value_f/function.der[var.name]
            key_val_dict[var] = var.val
        if verbose is True:
            print(f'Iteration at {nums_iteration}, at {list(key_val_dict.values())} ')
        if abs(value_f) <= threshold:
            break
        nums_iteration +=1

In [20]:
x = Variable('x', 3)
y = Variable('y', 2)
f_x_y = x**2 + y**2
list_var = [x, y]
newton_method_scalar(f_x_y, list_var, 1e-6, 50, verbose=True)

Iteration at 1, at [0.8333333333333335, -1.25] 
Iteration at 2, at [-0.520833333333333, -0.3472222222222221] 
Iteration at 3, at [-0.14467592592592576, 0.21701388888888873] 
Iteration at 4, at [0.09042245370370375, 0.06028163580246912] 
Iteration at 5, at [0.025117348251028848, -0.0376760223765433] 
Iteration at 6, at [-0.01569834265689307, -0.010465561771262042] 
Iteration at 7, at [-0.004360650738025858, 0.006540976107038788] 
Iteration at 8, at [0.002725406711266163, 0.0018169378075107746] 
Iteration at 9, at [0.0007570574197961568, -0.0011355861296942354] 
Iteration at 10, at [-0.0004731608873725984, -0.0003154405915817322] 
Iteration at 11, at [-0.0001314335798257218, 0.00019715036973858274] 


In [21]:
def newton_method_scalar(function, list_variables, threshold, max_iter, verbose=True):
    nums_iteration = 1
    key_val_dict = {}
    while True:
        value_f = function.val
        for var in list_variables:
            var.val = var.val - value_f/function.der[var.name]
            key_val_dict[var] = var.val
        if verbose is True:
            print(f'Iteration at {nums_iteration}, at {list(key_val_dict.values())} ')
        if abs(value_f) <= threshold:
            break
        nums_iteration +=1

In [22]:
x = Variable('x', 3)
y = Variable('y', 2)
f_x_y = x**2 + y**2
list_var = [x, y]
newton_method_scalar(f_x_y, list_var, 1e-6, 50, verbose=True)

Iteration at 1, at [0.8333333333333335, -1.25] 
Iteration at 2, at [-1.333333333333333, -4.5] 
Iteration at 3, at [-3.4999999999999996, -7.75] 
Iteration at 4, at [-5.666666666666666, -11.0] 
Iteration at 5, at [-7.833333333333332, -14.25] 
Iteration at 6, at [-9.999999999999998, -17.5] 
Iteration at 7, at [-12.166666666666664, -20.75] 
Iteration at 8, at [-14.33333333333333, -24.0] 
Iteration at 9, at [-16.499999999999996, -27.25] 
Iteration at 10, at [-18.666666666666664, -30.5] 
Iteration at 11, at [-20.833333333333332, -33.75] 
Iteration at 12, at [-23.0, -37.0] 
Iteration at 13, at [-25.166666666666668, -40.25] 
Iteration at 14, at [-27.333333333333336, -43.5] 
Iteration at 15, at [-29.500000000000004, -46.75] 
Iteration at 16, at [-31.66666666666667, -50.0] 
Iteration at 17, at [-33.833333333333336, -53.25] 
Iteration at 18, at [-36.0, -56.5] 
Iteration at 19, at [-38.166666666666664, -59.75] 
Iteration at 20, at [-40.33333333333333, -63.0] 
Iteration at 21, at [-42.4999999999999

Iteration at 1040, at [-2250.3333333333476, -3378.0] 
Iteration at 1041, at [-2252.500000000014, -3381.25] 
Iteration at 1042, at [-2254.6666666666806, -3384.5] 
Iteration at 1043, at [-2256.833333333347, -3387.75] 
Iteration at 1044, at [-2259.0000000000136, -3391.0] 
Iteration at 1045, at [-2261.16666666668, -3394.25] 
Iteration at 1046, at [-2263.3333333333467, -3397.5] 
Iteration at 1047, at [-2265.500000000013, -3400.75] 
Iteration at 1048, at [-2267.6666666666797, -3404.0] 
Iteration at 1049, at [-2269.833333333346, -3407.25] 
Iteration at 1050, at [-2272.0000000000127, -3410.5] 
Iteration at 1051, at [-2274.1666666666792, -3413.75] 
Iteration at 1052, at [-2276.3333333333458, -3417.0] 
Iteration at 1053, at [-2278.5000000000123, -3420.25] 
Iteration at 1054, at [-2280.666666666679, -3423.5] 
Iteration at 1055, at [-2282.8333333333453, -3426.75] 
Iteration at 1056, at [-2285.000000000012, -3430.0] 
Iteration at 1057, at [-2287.1666666666783, -3433.25] 
Iteration at 1058, at [-228

Iteration at 1769, at [-3829.833333333237, -5747.25] 
Iteration at 1770, at [-3831.9999999999036, -5750.5] 
Iteration at 1771, at [-3834.16666666657, -5753.75] 
Iteration at 1772, at [-3836.3333333332366, -5757.0] 
Iteration at 1773, at [-3838.499999999903, -5760.25] 
Iteration at 1774, at [-3840.6666666665697, -5763.5] 
Iteration at 1775, at [-3842.833333333236, -5766.75] 
Iteration at 1776, at [-3844.9999999999027, -5770.0] 
Iteration at 1777, at [-3847.166666666569, -5773.25] 
Iteration at 1778, at [-3849.3333333332357, -5776.5] 
Iteration at 1779, at [-3851.4999999999022, -5779.75] 
Iteration at 1780, at [-3853.6666666665687, -5783.0] 
Iteration at 1781, at [-3855.8333333332353, -5786.25] 
Iteration at 1782, at [-3857.999999999902, -5789.5] 
Iteration at 1783, at [-3860.1666666665683, -5792.75] 
Iteration at 1784, at [-3862.333333333235, -5796.0] 
Iteration at 1785, at [-3864.4999999999013, -5799.25] 
Iteration at 1786, at [-3866.666666666568, -5802.5] 
Iteration at 1787, at [-3868

Iteration at 2404, at [-5205.666666666707, -7811.0] 
Iteration at 2405, at [-5207.833333333374, -7814.25] 
Iteration at 2406, at [-5210.000000000041, -7817.5] 
Iteration at 2407, at [-5212.166666666708, -7820.75] 
Iteration at 2408, at [-5214.333333333375, -7824.0] 
Iteration at 2409, at [-5216.500000000042, -7827.25] 
Iteration at 2410, at [-5218.666666666709, -7830.5] 
Iteration at 2411, at [-5220.833333333376, -7833.75] 
Iteration at 2412, at [-5223.000000000043, -7837.0] 
Iteration at 2413, at [-5225.16666666671, -7840.25] 
Iteration at 2414, at [-5227.333333333377, -7843.5] 
Iteration at 2415, at [-5229.500000000044, -7846.75] 
Iteration at 2416, at [-5231.666666666711, -7850.0] 
Iteration at 2417, at [-5233.833333333378, -7853.25] 
Iteration at 2418, at [-5236.000000000045, -7856.5] 
Iteration at 2419, at [-5238.1666666667115, -7859.75] 
Iteration at 2420, at [-5240.3333333333785, -7863.0] 
Iteration at 2421, at [-5242.5000000000455, -7866.25] 
Iteration at 2422, at [-5244.666666

Iteration at 3404, at [-7372.333333333677, -11061.0] 
Iteration at 3405, at [-7374.500000000344, -11064.25] 
Iteration at 3406, at [-7376.666666667011, -11067.5] 
Iteration at 3407, at [-7378.833333333678, -11070.75] 
Iteration at 3408, at [-7381.000000000345, -11074.0] 
Iteration at 3409, at [-7383.166666667012, -11077.25] 
Iteration at 3410, at [-7385.333333333679, -11080.5] 
Iteration at 3411, at [-7387.500000000346, -11083.75] 
Iteration at 3412, at [-7389.666666667013, -11087.0] 
Iteration at 3413, at [-7391.83333333368, -11090.25] 
Iteration at 3414, at [-7394.0000000003465, -11093.5] 
Iteration at 3415, at [-7396.1666666670135, -11096.75] 
Iteration at 3416, at [-7398.3333333336805, -11100.0] 
Iteration at 3417, at [-7400.500000000347, -11103.25] 
Iteration at 3418, at [-7402.666666667014, -11106.5] 
Iteration at 3419, at [-7404.833333333681, -11109.75] 
Iteration at 3420, at [-7407.000000000348, -11113.0] 
Iteration at 3421, at [-7409.166666667015, -11116.25] 
Iteration at 3422

...

Iteration at 10400, at [-22530.33333333494, -33798.0] 
Iteration at 10401, at [-22532.500000001608, -33801.25] 
Iteration at 10402, at [-22534.666666668276, -33804.5] 
Iteration at 10403, at [-22536.833333334944, -33807.75] 
Iteration at 10404, at [-22539.00000000161, -33811.0] 
Iteration at 10405, at [-22541.16666666828, -33814.25] 
Iteration at 10406, at [-22543.333333334947, -33817.5] 
Iteration at 10407, at [-22545.500000001615, -33820.75] 
Iteration at 10408, at [-22547.666666668283, -33824.0] 
Iteration at 10409, at [-22549.83333333495, -33827.25] 
Iteration at 10410, at [-22552.00000000162, -33830.5] 
Iteration at 10411, at [-22554.166666668287, -33833.75] 
Iteration at 10412, at [-22556.333333334955, -33837.0] 
Iteration at 10413, at [-22558.500000001623, -33840.25] 
Iteration at 10414, at [-22560.66666666829, -33843.5] 
Iteration at 10415, at [-22562.83333333496, -33846.75] 
Iteration at 10416, at [-22565.000000001626, -33850.0] 
Iteration at 10417, at [-22567.166666668294, -3

Iteration at 11400, at [-24697.00000000282, -37048.0] 
Iteration at 11401, at [-24699.166666669487, -37051.25] 
Iteration at 11402, at [-24701.333333336155, -37054.5] 
Iteration at 11403, at [-24703.500000002823, -37057.75] 
Iteration at 11404, at [-24705.66666666949, -37061.0] 
Iteration at 11405, at [-24707.83333333616, -37064.25] 
Iteration at 11406, at [-24710.000000002827, -37067.5] 
Iteration at 11407, at [-24712.166666669495, -37070.75] 
Iteration at 11408, at [-24714.333333336162, -37074.0] 
Iteration at 11409, at [-24716.50000000283, -37077.25] 
Iteration at 11410, at [-24718.6666666695, -37080.5] 
Iteration at 11411, at [-24720.833333336166, -37083.75] 
Iteration at 11412, at [-24723.000000002834, -37087.0] 
Iteration at 11413, at [-24725.166666669502, -37090.25] 
Iteration at 11414, at [-24727.33333333617, -37093.5] 
Iteration at 11415, at [-24729.500000002838, -37096.75] 
Iteration at 11416, at [-24731.666666669506, -37100.0] 
Iteration at 11417, at [-24733.833333336173, -3

Iteration at 12399, at [-26861.50000000403, -40294.75] 
Iteration at 12400, at [-26863.6666666707, -40298.0] 
Iteration at 12401, at [-26865.833333337367, -40301.25] 
Iteration at 12402, at [-26868.000000004035, -40304.5] 
Iteration at 12403, at [-26870.166666670702, -40307.75] 
Iteration at 12404, at [-26872.33333333737, -40311.0] 
Iteration at 12405, at [-26874.500000004038, -40314.25] 
Iteration at 12406, at [-26876.666666670706, -40317.5] 
Iteration at 12407, at [-26878.833333337374, -40320.75] 
Iteration at 12408, at [-26881.00000000404, -40324.0] 
Iteration at 12409, at [-26883.16666667071, -40327.25] 
Iteration at 12410, at [-26885.333333337378, -40330.5] 
Iteration at 12411, at [-26887.500000004045, -40333.75] 
Iteration at 12412, at [-26889.666666670713, -40337.0] 
Iteration at 12413, at [-26891.83333333738, -40340.25] 
Iteration at 12414, at [-26894.00000000405, -40343.5] 
Iteration at 12415, at [-26896.166666670717, -40346.75] 
Iteration at 12416, at [-26898.333333337385, -4

Iteration at 13399, at [-29028.16666667191, -43544.75] 
Iteration at 13400, at [-29030.333333338578, -43548.0] 
Iteration at 13401, at [-29032.500000005246, -43551.25] 
Iteration at 13402, at [-29034.666666671914, -43554.5] 
Iteration at 13403, at [-29036.83333333858, -43557.75] 
Iteration at 13404, at [-29039.00000000525, -43561.0] 
Iteration at 13405, at [-29041.166666671917, -43564.25] 
Iteration at 13406, at [-29043.333333338585, -43567.5] 
Iteration at 13407, at [-29045.500000005253, -43570.75] 
Iteration at 13408, at [-29047.66666667192, -43574.0] 
Iteration at 13409, at [-29049.83333333859, -43577.25] 
Iteration at 13410, at [-29052.000000005257, -43580.5] 
Iteration at 13411, at [-29054.166666671925, -43583.75] 
Iteration at 13412, at [-29056.333333338593, -43587.0] 
Iteration at 13413, at [-29058.50000000526, -43590.25] 
Iteration at 13414, at [-29060.66666667193, -43593.5] 
Iteration at 13415, at [-29062.833333338596, -43596.75] 
Iteration at 13416, at [-29065.000000005264, -

KeyboardInterrupt: 